In [5]:
import numpy as np  # arithmetic
import glob  # file searching
from astropy.io import fits  # work with fits files
import sys
np.set_printoptions(threshold=sys.maxsize)  # print full array values

In [7]:
def bright_line1d(filepattern, refim):
    # a function that calculates fluxes on a spectra by spectra basis.
    filelist = glob.glob(filepattern)  # gather files given an input filepattern
    ref = glob.glob(refim)  # set our reference image = input ref image
    
    # open our files with write permissions:
    allpout = open('fiber_per.out', 'w+')
    allrout = open('fiber_ratio.out', 'w+')
    mtpout = open('fiber_mtp.out', 'w+')
    out = open('output.out', 'w+')  # may delete this later as it's in another line.
    
    # a for-loop that loops over the number of elements in our filelist.
    # it takes an image for each index, then starts an if-statement.
    for i in range(0, len(filelist)):
        image = fits.getdata(filelist[i], ext=1).astype(np.int32)  # gather the fits image data for each index
        refimage = fits.getdata(refim).astype(np.int32)  # gather the fits image data of the the ref image. Does this need ext specification?
            
        # we have a few variables we will be working with.
        # python does not generally require an array initialization.
        # for now we will initialize them regardless.
        # the parameters are: flux, refflux, ratio, mtpratio, and mtpdiff.
        flux = np.zeros(299)
        refflux = np.zeros(299)
        perdiff = np.zeros(299)
        ratio = np.zeros(299)
        mtpratio = np.zeros(10)
        mtpdiff = np.zeros(10)
            
        # now, begin another set of loops that iterates over each of the 300 fibers.
        # we want to take the median across the x axis -- pixels -- y number of times (300) (right???)
        # read flux for each spectra:
        for j in range(0,299):  # j=0 is fiber #1 out of 300 fibers
            image[image < 0] = 0   # ignore negative fluxes
            # image[image > 18000] = 0  # ignore  cosmic ray hits, may need to change this value later
            flux[j] = np.median(image[299-j,:])  # fibers are reversed in 1d images compared to raw. Calculate the image fluxes
            
            refflux[j] = np.median(refimage[299-j,:])  # calculate the reference image flux
            ratio[j] = flux[j] / refflux[j]  # calculate the ratio per each flux and reference flux
            perdiff[j] = (flux[j] - refflux[j]) / refflux[j] * 100  # calculate the percent difference between the two
            out.write(str((j + 1) + flux[j] + refflux[j] + ratio[j] + perdiff[j])) # print to our output file

        sortmin = sorted(flux)  # sort the array of fluxes in order to get mins
        sortmax = sorted(flux, reverse=True)  # reverse the order to get the maxes
        sortfibermin = np.argsort(flux)  # sort the fiber number according to the sortmin and sortmax
        sortfibermax = np.argsort(flux)
        sortfibermax = sortfibermax[::-1] 

        print("Five Highest Flux Fibers", filelist[i])  # prints top three fluxes                
        print(sortfibermax[0]+1, ":", sortmax[0])  # we want to print the fiber number along with the flux associated.
        print(sortfibermax[1]+1, ":", sortmax[1])
        print(sortfibermax[2]+1, ":", sortmax[2])
        print(sortfibermax[3]+1, ":", sortmax[3])
        print(sortfibermax[4]+1, ":", sortmax[4])

        print("Five Lowest Flux Fibers", filelist[i])  # prints lowest three fluxes
        print(sortfibermin[0]+1, ":", sortmin[0])
        print(sortfibermin[1]+1, ":", sortmin[1])
        print(sortfibermin[2]+1, ":", sortmin[2])
        print(sortfibermin[3]+1, ":", sortmin[3])
        print(sortfibermin[4]+1, ":", sortmin[4])

        print("Median Ratio", np.median(ratio))
        print("Median Percent Difference", np.median(perdiff))
        print("Median Ratio Percent Difference")

        # for-loop to determine mtp ratio and difference
        for k in range(0, 10):
            mtpratio[k] = np.median(ratio[0 + 30 * k : 29 + 30 * k])
            mtpdiff[k] = np.median(perdiff[0 + 30 * k : 29 + 30 * k])
            print("MTP: ", k + 1, mtpratio[k], mtpdiff[k])

            # print machine-readable output
            allrout.write((filelist[i]) + str(ratio))
            allpout.write((filelist[i]) + str(perdiff))
            mtpout.write((filelist[i]) + str(mtpratio + mtpdiff))

    # close our files, we are done with them now
    mtpout.close()
    allrout.close()
    allpout.close()